In [1]:
import sys, os
sys.path.insert(0, '/home/llr/cms/wind/cmssw/CMSSW_9_4_2/src/ZZAnalysis/AnalysisStep/test/Python/')

In [2]:
from trainlib.ConfigFileHandler import ConfigFileHandler
from trainlib.ModelCollectionConfigFileHandler import ModelCollectionConfigFileHandler

Welcome to JupyROOT 6.10/09


Using TensorFlow backend.


In [3]:
from vis.losses import ActivationMaximization

In [4]:
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np

In [5]:
K.set_learning_phase(False)

In [6]:
def get_model(run_dir, model_name):
    confhandler = ModelCollectionConfigFileHandler()
    confhandler.load_configuration(os.path.join(run_dir, "settings.conf"))
    
    mcolls = confhandler.GetModelCollection(weightpath = os.path.join(run_dir, "training/"))
    
    for mcoll in mcolls:
        if model_name in mcoll.model_dict.keys():
            found_mcoll = mcoll
            model = mcoll.model_dict[model_name]
            pre = mcoll.preprocessor_dict[model_name]
            break
        
    return model, pre

In [7]:
# load the neural network that is to be visualized
run_dir = "/data_CMS/cms/wind/180508_bkg_cat_ZXonly/number_layers_2.0_number_neurons_64.0/"
model_name = "D_VBF_ggH_2j_ML"
#model_name = "D_VBF_WHh_ML"

In [8]:
model, pre = get_model(run_dir, model_name)

attempting to load configuration file from /data_CMS/cms/wind/180508_bkg_cat_ZXonly/number_layers_2.0_number_neurons_64.0/settings.conf
got the following list of hyperparams: {u'number_layers': 2.0, u'number_neurons': 64.0}
building network with 28 inputs
FlexiblePCAPreprocessor set up for 28 inputs
got the following list of hyperparams: {u'number_layers': 2.0, u'number_neurons': 64.0}
building network with 10 inputs
FlexiblePCAPreprocessor set up for 10 inputs
got the following list of hyperparams: {u'number_layers': 2.0, u'number_neurons': 64.0}
building network with 15 inputs
FlexiblePCAPreprocessor set up for 15 inputs
found the following models belonging to this collection:
D_VBF_ggH_2j_ML
D_VBF_ggH_0j_ML
D_VBF_ggH_1j_ML
now attempting to load model from file /data_CMS/cms/wind/180508_bkg_cat_ZXonly/number_layers_2.0_number_neurons_64.0/training/D_VBF_ggH_ML/D_VBF_ggH_2j_ML/final.hdf5
read weights from /data_CMS/cms/wind/180508_bkg_cat_ZXonly/number_layers_2.0_number_neurons_64.0/

2018-05-15 18:06:03.489515: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-05-15 18:06:03.489539: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-05-15 18:06:03.489547: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.


In [9]:
pre.processed_columns

Index([       u'JetPt_2',          u'PFMET', u'D_VBF2j_ZHh_ME',
       u'D_VBF2j_WHh_ME', u'D_VBF2j_ggH_ME',        u'JetPt_0',
               u'Z2Mass',       u'JetEta_1',           u'Z1Pt',
            u'ZZMassErr',        u'JetPt_1',   u'D_WHh_ggH_ME',
         u'D_ZHh_ggH_ME',         u'Z1Mass',           u'Z2Pt',
             u'JetEta_2',           u'ZZPt',   u'D_WHh_ZHh_ME',
                u'ZZEta',       u'JetEta_0',   u'JetPhi_2_sin',
         u'JetPhi_2_cos',   u'JetPhi_0_sin',   u'JetPhi_0_cos',
            u'ZZPhi_sin',      u'ZZPhi_cos',   u'JetPhi_1_sin',
         u'JetPhi_1_cos'],
      dtype='object')

In [10]:
keras_model = model.get_keras_model()

In [11]:
layer_dict = {layer.name: layer for layer in keras_model.layers}

In [12]:
layer_dict

{u'D_VBF_ggH_2j_ML_input': <keras.engine.topology.InputLayer at 0x7f8d90f11510>,
 'activation_1': <keras.layers.core.Activation at 0x7f8d90eb5650>,
 'activation_2': <keras.layers.core.Activation at 0x7f8d90e59d50>,
 'activation_3': <keras.layers.core.Activation at 0x7f8d90df74d0>,
 'batch_normalization_1': <keras.layers.normalization.BatchNormalization at 0x7f8d90eda410>,
 'batch_normalization_2': <keras.layers.normalization.BatchNormalization at 0x7f8d90e29dd0>,
 'batch_normalization_3': <keras.layers.normalization.BatchNormalization at 0x7f8d90d89750>,
 'dense_1': <keras.layers.core.Dense at 0x7f8d91d9c310>,
 'dense_2': <keras.layers.core.Dense at 0x7f8d90e3ff90>,
 'dense_3': <keras.layers.core.Dense at 0x7f8d90dae490>,
 'dropout_1': <keras.layers.core.Dropout at 0x7f8d90e8af50>,
 'dropout_2': <keras.layers.core.Dropout at 0x7f8d90daef90>,
 'dropout_3': <keras.layers.core.Dropout at 0x7f8d90d44c50>,
 'target': <keras.layers.core.Dense at 0x7f8d90e78cd0>}

In [13]:
def normalize(x):
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

In [14]:
layer_output = layer_dict["target"].output

In [15]:
inputs = keras_model.input

In [16]:
loss = K.mean(layer_output[0])

In [17]:
grads = K.gradients(loss, inputs)[0]
grads = normalize(grads)

In [18]:
iterate = K.function([inputs], [loss, grads])

In [19]:
stepsize = 0.1

In [20]:
def synthesize_input_from(cur_input, threshold):
    for i in range(200):
        cur_loss, cur_grads = iterate([cur_input])
        cur_input += stepsize * cur_grads

        if cur_loss > threshold:
            break
            
    pre_pca_inputs = pre.base_preprocessor.pca.inverse_transform(cur_input).flatten().tolist()
    synthesized_input = {key: val for (key, val) in zip(pre.processed_columns, pre_pca_inputs)}
    
    return synthesized_input

In [21]:
def synthesize_inputs(num_inputs, threshold):
    synthesized = []
    
    for i in range(num_inputs):
        # start from a random input
        cur_input = np.random.uniform(-1, 1, size = (1, keras_model.input_shape[1]))
        synthesized_input = synthesize_input_from(cur_input, threshold)
        
        synthesized.append(synthesized_input)
        
    return synthesized

In [22]:
def extract_column(input_dicts, colname):
    retval = [cur_dict[colname] for cur_dict in input_dicts]
    
    return retval

In [23]:
def make_activation_maximization_plot(x_column, y_column, x_label, y_label, x_range, y_range, threshold):
    synthesized = synthesize_inputs(500000, threshold = threshold)
    
    x_data = extract_column(synthesized, x_column)
    y_data = extract_column(synthesized, y_column)
    
    x_edges = np.linspace(*x_range)
    y_edges = np.linspace(*y_range)
    
    H, x_edges, y_edges = np.histogram2d(x_data, y_data, bins = (x_edges, y_edges))
    
    fig = plt.figure(figsize = (5, 5))
    ax = fig.add_subplot(111)

    cax = ax.imshow(H, interpolation = "nearest", cmap = "hot", origin = "low", 
                    extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]])
    
    ax.set_xlabel(x_label, fontsize = 17)
    ax.set_ylabel(y_label, fontsize = 17)

    #fig.colorbar(cax)
    plt.tight_layout()

    return fig

In [24]:
outpath = "/data_CMS/cms/wind/ActivationMaximization/"

In [25]:
fig = make_activation_maximization_plot("D_VBF2j_ggH_ME", "D_VBF2j_ZHh_ME", 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ggH}}^{\mathrm{ME}}$', 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ZH-hadr}}^{\mathrm{ME}}$', 
                                        (0, 1, 60), (0, 1, 60), threshold = 0.3)
plt.savefig(os.path.join(outpath, "D_VBF_ggH_MELA_03.pdf"))

In [26]:
fig = make_activation_maximization_plot("D_VBF2j_ggH_ME", "D_VBF2j_ZHh_ME", 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ggH}}^{\mathrm{ME}}$', 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ZH-hadr}}^{\mathrm{ME}}$', 
                                        (0, 1, 60), (0, 1, 60), threshold = 0.5)
plt.savefig(os.path.join(outpath, "D_VBF_ggH_MELA_05.pdf"))

In [27]:
fig = make_activation_maximization_plot("D_VBF2j_ggH_ME", "D_VBF2j_ZHh_ME", 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ggH}}^{\mathrm{ME}}$', 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ZH-hadr}}^{\mathrm{ME}}$', 
                                        (0, 1, 60), (0, 1, 60), threshold = 0.7)
plt.savefig(os.path.join(outpath, "D_VBF_ggH_MELA_07.pdf"))

In [28]:
fig = make_activation_maximization_plot("D_VBF2j_ggH_ME", "D_VBF2j_ZHh_ME", 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ggH}}^{\mathrm{ME}}$', 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ZH-hadr}}^{\mathrm{ME}}$', 
                                        (0, 1, 60), (0, 1, 60), threshold = 0.9)
plt.savefig(os.path.join(outpath, "D_VBF_ggH_MELA_09.pdf"))

In [27]:
fig = make_activation_maximization_plot("JetEta_0", "JetEta_1", r'$\eta_{\,\,0}$', r'$\eta_{\,\,1}$', (-6, 6, 60), (-6, 6, 60), threshold = 0.7)
plt.savefig(os.path.join(outpath, "D_VBF_WHh_07.pdf"))

In [28]:
fig = make_activation_maximization_plot("JetEta_0", "JetEta_1", r'$\eta_{\,\,0}$', r'$\eta_{\,\,1}$', (-6, 6, 60), (-6, 6, 60), threshold = 1.0 - 1e-1)
plt.savefig(os.path.join(outpath, "D_VBF_WHh_09.pdf"))

In [29]:
fig = make_activation_maximization_plot("JetEta_0", "JetEta_1", r'$\eta_{\,\,0}$', r'$\eta_{\,\,1}$', (-6, 6, 60), (-6, 6, 60), threshold = 1.0 - 1e-2)
plt.savefig(os.path.join(outpath, "D_VBF_WHh_099.pdf"))

In [30]:
fig = make_activation_maximization_plot("JetEta_0", "JetEta_1", r'$\eta_{\,\,0}$', r'$\eta_{\,\,1}$', (-6, 6, 60), (-6, 6, 60), threshold = 1.0 - 1e-3)
plt.savefig(os.path.join(outpath, "D_VBF_WHh_0999.pdf"))